In [13]:
import pandas as pd
from tvDatafeed import TvDatafeed, Interval

In [14]:
tv = TvDatafeed()
dax = tv.get_hist("DAX", "XETR", Interval.in_30_minute, n_bars=5000)

you are using nologin method, data you access may be limited


In [21]:
df = dax.drop(["symbol", "volume"], axis=1).reset_index()

In [33]:

def first_prep(df):
    from datetime import time
    from functools import reduce

    df["datetime"] = pd.to_datetime(df.datetime)
    df["date"] = df.datetime.dt.date


    #TODO: turn times into parameters
    open_time = time(9, 0)
    close_time = time(17, 30)
    ibclose_time = time(10, 0)

    daily_open = df[df.datetime.dt.time == open_time][["date", "open"]]
    daily_close = df[df.datetime.dt.time == close_time][["date", "close"]]
    ib_close = df[df.datetime.dt.time == ibclose_time][["date", "close"]].rename(
        columns={"close" : "ibclose"}
    )
    daily_low = df.groupby("date").low.min().reset_index()
    daily_high = df.groupby("date").high.max().reset_index()

    ib_low = df[df.datetime.dt.time <= ibclose_time].groupby("date").low.min().reset_index().rename(columns={"low" : "iblow"})
    ib_high = df[df.datetime.dt.time <= ibclose_time].groupby("date").high.max().reset_index().rename(columns={"high" : "ibhigh"})

    data = reduce(
        lambda l, r: pd.merge(l, r, on="date"),
        [daily_open, daily_high, daily_low, ib_low, ib_high, ib_close]
    )

    return data


    

In [34]:
first_prep(df)

,date,open,high,low,iblow,ibhigh,ibclose
0,2021-03-10,14439.45,14560.97,14408.52,14408.52,14488.46,14477.09
1,2021-03-11,14553.54,14595.02,14515.07,14521.89,14578.46,14528.80
2,2021-03-12,14492.45,14527.42,14450.72,14469.48,14527.42,14484.69
3,2021-03-15,14504.49,14572.24,14409.35,14500.52,14572.24,14544.67
4,2021-03-16,14519.72,14601.79,14511.30,14511.30,14550.42,14550.27
...,...,...,...,...,...,...,...
273,2022-04-04,14503.96,14535.07,14334.10,14334.10,14515.79,14382.35
274,2022-04-05,14504.80,14603.44,14347.33,14471.09,14603.44,14563.51
275,2022-04-06,14402.16,14437.91,14027.18,14216.42,14437.91,14245.02
276,2022-04-07,14187.13,14315.35,14069.53,14123.05,14234.65,14155.22
